## Importing libraries

In [1]:
import sys
import json
sys.path.insert(0, "./nanohubtools") ## Under develpoment branch
#from nanohubtools import *
import nanohub.uidl.teleport as t
from nanohub.uidl.rappture import RapptureBuilder
from nanohub.uidl.material import MaterialBuilder
from nanohub.uidl.material import MaterialContent
from nanohub.uidl.plotly import PlotlyBuilder
from nanohub.uidl.app import AppBuilder

from nanohub.uidl.nanohub import Auth
TOOLNAME="pntoy"
STATE_LOADER_STATUS = "loader_status"
STATE_LOADER_OPEN = "loader_open"
STATE_LOGIN_OPEN = "login_open"
STATE_ERROR_STATUS = "error_status"
STATE_ERROR_OPEN = "error_open"

## Creating the project and adding GUI variables 

In [2]:
Project = t.TeleportProject("PNToy Lab")
Component = Project.root
Component.addStateVariable("DrawerIsVisible", {"type":"boolean", "defaultValue": True})
Component.addStateVariable("AppBarStyle", {"type":"string", "defaultValue": "bar_shifted"})
Component.addStateVariable("open_results", {"type":"boolean", "defaultValue": False})
Component.addStateVariable("open_params", {"type":"boolean", "defaultValue": True})

## Getting session information to request data from webservices

In [3]:
import os
auth_data = {
    'grant_type' : 'tool',
}
with open(os.environ["SESSIONDIR"]+"/resources") as file:
    lines = [line.split(" ", 1) for line in file.readlines()]
    properties = {line[0].strip(): line[1].strip() for line in lines if len(line)==2}
    auth_data["sessiontoken"] = properties["session_token"]
    auth_data["sessionnum"] = properties["sessionid"] 

## Creating Error

In [4]:
ErrorMessage = RapptureBuilder.Error( 
    Component, 
    error_status = STATE_ERROR_STATUS,
    error_open = STATE_ERROR_OPEN,
    is_open = False
)

## Creating loader

In [5]:
Loader = RapptureBuilder.Loader( 
    Component, 
    loader_status = STATE_LOADER_STATUS,
    loader_open = STATE_LOADER_OPEN,
    is_open = False
)

## Schema

In [6]:
RapptureBuilder.buildSchema(
    Project,
    Component,
    url = "https://nanohub.org/api/tools",
    toolname = TOOLNAME
)
Component.addPropVariable("onLoadSchema", {"type":"func", 'defaultValue' : '(e)=>{e.setState({"'+ STATE_LOADER_OPEN +'":false})}'})    

## Authentication

In [7]:
Login, CLogin = Auth.Login(
    Project,
    Component,
    client_id = "4135f6de1fad82002813c7a796ff983e",
    client_secret = "286a25a4f3346243d3d3df265945302b7095f473",
    #url = "https://dev.nanohub.org/api/querycache/squidlog/proxy?url=https://nanohub.org/api/developer/oauth/token",   
    url = "https://nanohub.org/api/developer/oauth/token",   
    open_state = STATE_LOGIN_OPEN
)
#Login.content.events["onError"]=[
#    { "type": "stateChange", "modifies": STATE_ERROR_OPEN, "newState": True},
#    { "type": "stateChange", "modifies": STATE_ERROR_STATUS, "newState": '$e'},
#]
Login.content.events["onAuth"] = [
    { "type": "stateChange", "modifies": STATE_LOGIN_OPEN, "newState": False},
    { "type": "propCall2", "calls": "buildSchema", "args": ['self'] }
]

In [8]:
Drawer = MaterialBuilder.Drawer(Project, state="DrawerIsVisible", variant="persistent" )

In [9]:
import nanohub.remote as nr
tool = nr.Tools(auth_data)
params = tool.getToolParameters(TOOLNAME)
SETTINGS = {}
for option, value in params.items():
    if isinstance(value,nr.params.Number):
        SETTINGS[option] = {
            "type": "Number",
            "default_value": value.default, 
            "units" : value.units, 
            "min" : value.min, 
            "max" : value.max, 
            "label" : value.label,
            "description" : value.description,
        }   
    elif isinstance(value,nr.params.Integer):
        SETTINGS[option] = {
            "type": "Integer",
            "default_value": value.default, 
            "units" : value.units, 
            "min" : value.min, 
            "max" : value.max, 
            "label" : value.label,
            "description" : value.description,
        }   
    elif isinstance(value,nr.params.String):
        SETTINGS[option] = {
            "type": "String",
            "default_value": value.default, 
            "label" : value.label,
            "description" : value.description,
        }   
    elif isinstance(value,nr.params.Choice):
        SETTINGS[option] = {
            "type": "Select",
            "default_value": value.default, 
            "options": {k[1] : k[0] for k in value.options}, 
            "units" : value.units, 
            "label" : value.label,
            "description" : value.description,
        }
    elif isinstance(value,nr.params.Boolean):
        SETTINGS[option] = {
            "type": "Boolean",
            "default_value": (value.default == "yes"),
            "description" : value.description,
            "label" : value.label,
        }
#SETTINGS        

## Defining Layout

In [10]:
LAYOUT = tool.getToolLayout(TOOLNAME)
#LAYOUT

## Creating the settings panel

In [11]:
#url_sim = "https://dev.nanohub.org/api/querycache/squidlog/proxy?url=https://nanohub.org/api/tools"
url_sim = "https://nanohub.org/api/tools"
PNTOYSettings = AppBuilder.Settings(
    Project,
    Component, 
    SETTINGS,
    url=url_sim,
    toolname = TOOLNAME,
    layout = LAYOUT['input']
)
PNTOYSettings.content.events["onError"]=[
    { "type": "stateChange", "modifies": STATE_LOADER_OPEN,"newState": False}
]
PNTOYSettings.content.events["click"]=[
    { "type": "stateChange", "modifies": STATE_LOADER_OPEN,"newState": True}
]
PNTOYSettings.content.events["change"]=[]

PNTOYSettings.content.events["submit"] = [
    { "type": "stateChange", "modifies": "parameters","newState": '$e.target.value'}
]

PNTOYSettings.content.events["onStatusChange"]=[
    { "type": "stateChange", "modifies": STATE_LOADER_STATUS,"newState": "$e.target.value"}
]
PNTOYSettings.content.events["onSuccess"]=[
  { "type": "stateChange", "modifies": "open_plot","newState":{'s1':'primary'} },
  { "type": "propCall2", "calls": "loadSequence","args": ["self", "['s1']"]},
  { "type": "stateChange", "modifies": STATE_LOADER_OPEN,"newState": False },  
  { "type": "stateChange", "modifies": "open_results","newState": True},
  { "type": "stateChange", "modifies": "open_params","newState": False},
]

## Creating the output options

In [12]:
RapptureBuilder.loadXY(Project, Component)
RapptureBuilder.loadSequence(Project, Component)
RESULTS = {
    "bands" : {
        'title' : 'Energy Band Diagram',
        'action' : { "type": "propCall2", "calls": "loadSequence","args": ["self", "['s1']" ] },
    },
    "cv" : {
        'title' : 'C-V Characteristics',
        'action' : { "type": "propCall2", "calls": "loadXY","args": ["self", "['cap']" ] },
    },
    "iv" : {
        'title' : 'I-V Characteristics',
        'action' : { "type": "propCall2", "calls": "loadXY","args": ["self", "['iv']" ] },
    },
    "current" : {
        'title' : 'Electron and Hole Current',
        'action' : { "type": "propCall2", "calls": "loadSequence","args": ["self", "['s0']" ] },
    },
    "density" : {
        'title' : 'Electron and Hole Density',
        'action' : { "type": "propCall2", "calls": "loadSequence","args": ["self", "['s2']" ] },
    },
    "net" : {
        'title' : 'Net Charge Density',
        'action' : { "type": "propCall2", "calls": "loadSequence","args": ["self", "['s4']" ] },
    },
    "potential" : {
        'title' : 'Electrostatic Potential',
        'action' : { "type": "propCall2", "calls": "loadSequence","args": ["self", "['s5']" ] },
    },
    "field" : {
        'title' : 'Electric Field',
        'action' : { "type": "propCall2", "calls": "loadSequence","args": ["self", "['s6']" ] },
    },
    "recombination" : {
        'title' : 'Recombination Rate',
        'action' : { "type": "propCall2", "calls": "loadSequence","args": ["self", "['s7']" ] },
    },
    "carrier" : {
        'title' : 'Excess Carrier Density',
        'action' : { "type": "propCall2", "calls": "loadSequence","args": ["self", "['s3']" ] },
    },
}

PNTOYResults = AppBuilder.Results( 
    Component,
    results = RESULTS,
    onClick = [{ "type": "stateChange", "modifies": STATE_LOADER_OPEN,"newState": True }],
    onLoad = [{ "type": "stateChange", "modifies": STATE_LOADER_OPEN,"newState": False }],
)


## Creating Colorschema and adding nanoHUB logo

In [13]:
ThemeProvider = MaterialBuilder.ThemeProvider( Component, MaterialBuilder.DefaultTheme(
    primary_color = '#699FBB',
    secondary_color = '#f1f1f1',
    primary_bg = '#FFFFFF',
    secondary_bg = '#dbeaf0',
    default_button = 'rgba(255, 255, 255, 0.87)',
    primary_button = 'rgba(255, 255, 255, 0.87)',
    secondary_button = 'rgba(0, 0, 0, 0.87)',
    default_button_bg = 'rgb(63, 162, 192)',
    primary_button_bg = 'rgba(0, 0, 0, 0.65)',
    secondary_button_bg = 'rgba(0, 0, 0, 0.12)',
))
AppBar = MaterialBuilder.AppBar(state="DrawerIsVisible", styles=("AppBarStyle", ["bar_shifted", "bar_normal"]), title="PNToy lab")
logo = t.TeleportElement(t.TeleportContent(elementType="img"))
logo.content.attrs["width"] = "120"
logo.content.attrs["src"] = "https://nanohub.org/app/site/media/images/PressKit/nanoHUB_logo_color.jpg"
AppBar.content.children[0].addContent(logo)


## Creating Expansion panels

In [14]:
#Project.globals.assets.append({"type": "style", "content": ".bar_shifted {margin-left:350px; width: calc(100% - 350px);}"})
Project.globals.assets.append({"type": "style", "content": ".bar_shifted {margin-left:0px;}"})
Project.globals.assets.append({ "type": "style", "content": ".bar_normal {margin-left:0px}"})
ExpansionPanel = MaterialBuilder.ExpansionPanel(
    title="Results", 
    disabled=False,
    content=[PNTOYResults],
    expanded={
      "type": "dynamic",
      "content": {
        "referenceType": "state",
        "id": "open_results"
      }    
    },     
)
ExpansionPanel.content.events['change'] = [{ "type": "stateChange", "modifies": "open_results","newState": "$toggle"}]
Drawer.addContent(ExpansionPanel)

ParametersPanel = MaterialBuilder.ExpansionPanel(
    title="Parameters", 
    disabled=False, 
    expanded={
      "type": "dynamic",
      "content": {
        "referenceType": "state",
        "id": "open_params"
      }    
    }, 
    content=[PNTOYSettings]
)
ParametersPanel.content.events['change'] = [{ "type": "stateChange", "modifies": "open_params","newState": "$toggle"}]
Drawer.addContent(ParametersPanel)


## Creating reset button

In [15]:
Gridt = t.TeleportElement(MaterialContent(elementType="Grid"))
Gridt.content.attrs["color"] = "secondary"
Gridt.content.attrs["container"] = True
Gridt.content.attrs["direction"] = "column"
resetSettings = {}
for k,v in SETTINGS.items():
    resetSettings[k] = v["default_value"]
Buttontt= MaterialBuilder.Button(
      title = "Reset Setting", 
      variant = "text", 
      style = {'backgroundColor':'#999999', 'borderRadius' : '0px', 'minHeight':'40px', 'color':'rgba(255, 255, 255, 0.87)'}, 
      onClickButton=[{ "type": "stateChange", "modifies": "parameters","newState": resetSettings}]
  )
Gridt.addContent(Buttontt)
Drawer.addContent(Gridt)

## Creationg ploting component

In [16]:
BasePlot = PlotlyBuilder.BasePlot( Project, Component, style_state="AppBarStyle")
BasePlot.content.style = {
  "height": "calc(100vh - 80px)",
  "width": "100%",
}

## Assembling the app

In [17]:
Gridh = t.TeleportElement(MaterialContent(elementType="Grid"))
Gridh.content.attrs["container"] = True
Gridh.content.attrs["direction"] = "row" 
Gridh.addContent(Drawer)
Gridh.addContent(BasePlot)  

Gridv = t.TeleportElement(MaterialContent(elementType="Grid"))
Gridv.content.attrs["container"] = True
Gridv.content.attrs["direction"] = "column"
Gridv.addContent(AppBar)
Gridv.addContent(Gridh)


ThemeProvider.addContent(Gridv)
ThemeProvider.addContent(Loader)
ThemeProvider.addContent(Login)

Component.addNode(ThemeProvider)
Project.buildReact( TOOLNAME+auth_data["sessiontoken"]+".html");

## Displaying the app in jupyter

In [18]:
from IPython.core.display import display
from IPython.display import IFrame
display(IFrame(src=TOOLNAME+auth_data["sessiontoken"]+".html", width="100%", height="800"))